In [9]:
!pip install psycopg2-binary
!pip install geopy

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import psycopg2
from geopy.distance import geodesic

In [13]:
# Seaborn 스타일 설정
sns.set_theme(style="whitegrid")

# ✅ PostgreSQL 연결 정보
DB_HOST = "14.34.134.108"   # DB ip 주소로 변경해야 함
DB_PORT = "5432"
DB_NAME = "ais_data"
DB_USER = "postgres"
DB_PASS = "ky76018500"

# 엔진 생성
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

## 파티션 별 데이터 개수와 선박 수 ( 실행속도 너무 길어 확인 어려움 -> 10000개 샘플링해서 확인)

In [ ]:
# 분석할 테이블 리스트
tables = ["ais_2020_02", "ais_2020_03", "ais_2020_04", "ais_2020_05", "ais_2020_06", "ais_2020_07", "ais_2020_08", "ais_2020_09", "ais_2020_10", "ais_2020_11", "ais_2020_12"]

results = []

for table in tables:
    query = f"""
        SELECT 
            '{table}' AS table_name,
            COUNT(*) AS total_rows,
            COUNT(DISTINCT mmsi) AS unique_ships
        FROM {table};
    """
    df = pd.read_sql(query, engine)
    results.append(df)

df_counts = pd.concat(results, ignore_index=True)
display(df_counts)

In [15]:
# 샘플링 기반 행 수 & 고유 MMSI 수 분석
tables = ["ais_2020_02", "ais_2020_03", "ais_2020_04", "ais_2020_05", "ais_2020_06", "ais_2020_07", "ais_2020_08", "ais_2020_09", "ais_2020_10", "ais_2020_11", "ais_2020_12"]

sample_stats = []

for table in tables:
    query = f"""
        SELECT 
            '{table}' AS table_name,
            COUNT(*) AS sample_rows,
            COUNT(DISTINCT mmsi) AS unique_ships
        FROM (
            SELECT * FROM {table} LIMIT 100000
        ) AS sample;
    """
    df = pd.read_sql(query, engine)
    sample_stats.append(df)

df_sample_counts = pd.concat(sample_stats, ignore_index=True)
print("샘플별 행 수 및 선박 수:")
display(df_sample_counts)

샘플별 행 수 및 선박 수:


,table_name,sample_rows,unique_ships
0,ais_2020_02,100000,15
1,ais_2020_03,100000,87
2,ais_2020_04,100000,278
3,ais_2020_05,100000,462
4,ais_2020_06,100000,901
5,ais_2020_07,100000,101
6,ais_2020_08,0,0
7,ais_2020_09,0,0
8,ais_2020_10,0,0
9,ais_2020_11,0,0


## 결측치 확인 ( 실행속도 너무 길어 확인 어려움 -> 10000개 샘플링해서 확인)

In [ ]:
null_results = []

for table in tables:
    query = f"""
        SELECT 
            '{table}' AS table_name,
            SUM(CASE WHEN mmsi IS NULL THEN 1 ELSE 0 END) AS mmsi_nulls,
            SUM(CASE WHEN timestamp IS NULL THEN 1 ELSE 0 END) AS timestamp_nulls,
            SUM(CASE WHEN latitude IS NULL THEN 1 ELSE 0 END) AS lat_nulls,
            SUM(CASE WHEN longitude IS NULL THEN 1 ELSE 0 END) AS lon_nulls,
            SUM(CASE WHEN sog IS NULL THEN 1 ELSE 0 END) AS sog_nulls,
            SUM(CASE WHEN cog IS NULL THEN 1 ELSE 0 END) AS cog_nulls,
            SUM(CASE WHEN heading IS NULL THEN 1 ELSE 0 END) AS heading_nulls
        FROM {table};
    """
    df = pd.read_sql(query, engine)
    null_results.append(df)

df_nulls = pd.concat(null_results, ignore_index=True)
display(df_nulls)

In [17]:
null_results = []

for table in tables:
    query = f"""
        SELECT 
            '{table}' AS table_name,
            SUM(CASE WHEN mmsi IS NULL THEN 1 ELSE 0 END) AS mmsi_nulls,
            SUM(CASE WHEN timestamp IS NULL THEN 1 ELSE 0 END) AS timestamp_nulls,
            SUM(CASE WHEN latitude IS NULL THEN 1 ELSE 0 END) AS lat_nulls,
            SUM(CASE WHEN longitude IS NULL THEN 1 ELSE 0 END) AS lon_nulls,
            SUM(CASE WHEN sog IS NULL THEN 1 ELSE 0 END) AS sog_nulls,
            SUM(CASE WHEN cog IS NULL THEN 1 ELSE 0 END) AS cog_nulls,
            SUM(CASE WHEN heading IS NULL THEN 1 ELSE 0 END) AS heading_nulls
        FROM (
            SELECT * FROM {table} LIMIT 10000
        ) AS sampled;
    """
    df = pd.read_sql(query, engine)
    null_results.append(df)

# 결과 통합
df_nulls = pd.concat(null_results, ignore_index=True)
display(df_nulls)

,table_name,mmsi_nulls,timestamp_nulls,lat_nulls,lon_nulls,sog_nulls,cog_nulls,heading_nulls
0,ais_2020_02,0,0,0,0,0,0,0
1,ais_2020_03,0,0,0,0,0,0,0
2,ais_2020_04,0,0,0,0,0,0,0
3,ais_2020_05,0,0,0,0,0,0,0
4,ais_2020_06,0,0,0,0,0,0,0
5,ais_2020_07,0,0,0,0,0,0,0
6,ais_2020_08,None,None,None,None,None,None,None
7,ais_2020_09,None,None,None,None,None,None,None
8,ais_2020_10,None,None,None,None,None,None,None
9,ais_2020_11,None,None,None,None,None,None,None
